In [1]:
#Python Initiation
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
import pandas as pd
import ipywidgets as widgets
import time
import math
from IPython.display import display

#Graph Formatting
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 14
fig_size[1] = 10
plt.rcParams["figure.figsize"] = fig_size
Res=100 #Resoultion of the calulations

#Graph Font Edit
font = {'family':'Times New Roman',
        'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)


In [4]:
SIAltTable=[i*1000 for i in [0,1,3,5,10,25,50,75,100,130,160,200,300,400,600,1000]]
SITempTable=[288.15,281.651,268.65,255.65,223.252,221.552,270.65,206.65,195.08,469.27,696.29,845.56,976.01,995.83,999.85,1000]
SIPressTable=[i*101325 for i in [1,0.887,0.66919,0.53313,0.26151,0.025158,0.00078735,0.000020408,0.00000031593,1.2341e-8,2.9997e-9,8.3628e-10,8.6557e-11,1.4328e-11,8.1056e-13,7.4155e-14]]
USAltTable=[i*3.28084 for i in SIAltTable]
USTempTable=[i*1.8 for i in SITempTable]
USPressTable=[i*0.020885434273039 for i in SIPressTable]

AltTable=SIAltTable
TempTable=SITempTable
PressTable=SIPressTable

def ChangeUnits(change):
    
    global AltTable
    global TempTable
    global PressTable
    global GasConstant
    
    if (Units_Select.value == 'Metric'): 
        gravity.value=9.81
        AltTable=SIAltTable
        TempTable=SITempTable
        PressTable=SIPressTable
        Alt_Select.options=AltTable

    elif (Units_Select.value == 'Imperial'):
        gravity.value=32.17
        AltTable=USAltTable
        TempTable=USTempTable
        PressTable=USPressTable
        Alt_Select.options=AltTable
                                  
    else:
        print('ERROR')
    
    
    
def UpdateValues(change):
    
        temperature3.value=TempTable[AltTable.index(Alt_Select.value)]
        pressure3.value=PressTable[AltTable.index(Alt_Select.value)]
        
        
def ChamberPressure(pambient,k,M):
    po=pambient*(1+0.5*(k-1)*M**2)**(k/(k-1))
    return po

def ExpansionRatio(k,M):
    ER=(1/M)*((1+0.5*(k-1)*M**2)/(0.5*(k+1)))**((0.5*(k+1))/(k-1))
    return ER

def AreaRatio(k,P1_P2):
    ε=((((k+1)/2)**(1/(k-1)))*((1/P1_P2)**(1/k))*np.sqrt(((k+1)/(k-1))*(1-(1/P1_P2)**((k-1)/k))))**(-1)
    return ε

def AreaRatio(M2,γ):
    ε=(1/M2)*np.sqrt(((1+(γ-1)*M2**2)/((γ+1)/2))**((γ+1)/(γ-1)))
    return ε

def PressureRatio(k,M):
    PR=((1+0.5*(k-1)*M**2)**(k/(k-1)))**(-1)
    return PR

def PropulsionRatio(Mo,Mf,Ms):
    PPR=(Mf-Ms)/(Mo-Ms)
    return PPR

def PropellentMassFraction (Md,Mo,Ms):
    ζm=Md/(Mo-Ms)
    return ζm 

def TempRatio(k,M):
    TR=((1+0.5*(k-1)*M**2))**(-1)
    return TR

def PropMass(InitialM , FinalM):
    m=InitialM-FinalM
    return m

def MRVehicle(InitialM , FinalM):
    mr=FinalM/InitialM
    return mr

def MRSubSys(InitialM , FinalM , NonSysMass):
    mr=(FinalM-NonSysMass)/(InitialM-NonSysMass)
    return mr

def PMFVehicle(PropellantMass , InitialM):
    PMF=PropellantMass/InitialM
    return PMF

def PMFSubSys(PropellantMass , InitialM , NonSysMass):
    PMF=PropellantMass/(InitialM-NonSysMass)
    return PMF

def MassFlowRatio(PropM , BurnTime):
    mdot=PropM/BurnTime
    return mdot

def WeightFlow(mdot , gravitySL):  
    WFR=mdot*gravitySL  
    return WFR

def EEV_ISP(Isp , Gravity):
    c=Isp*Gravity
    return c

def EEV_Force(Thrust , mdot):    
    c=Thrust/mdot    
    return c

def Thrust(At,vt,v2,Vt,p2,p3,A2):
    F=((At*vt*v2)/Vt)+(p2-p3)*A2
    return F

def Thrust(Cf,At,P1):
    F=Cf*At*P1
    return F

def IdealThrust(c , mdot):
    Fi=mdot*c
    return Fi

def MomentumThrust(mdot , V2):    
    Fm=mdot*V2    
    return Fm

def PressureThrust (P2,P3,A2):
    Fp=(P2-P3)*A2
    return Fp

def TotalThrust(MomentumThrust , PressureThrust):    
    Ft=MomentumThrust+PressureThrust    
    return Ft

def TotalImpulse(Thrust,Time):    
    It=Thrust*Time   
    return It

def Acceleration(Thrust , Mass):   
    a=Thrust/Mass    
    return a

def CStar(ChamberP , ThroatArea , mdot):    
    cstar=(ChamberP*ThroatArea)/mdot    
    return cstar

def SpecificImpulse(Thrust,WFR):    
    Isp=Thrust/WFR    
    return Isp

def SpecificImpulseV(V,SpecGrav):    
    Isp=V/SpecGrav    
    return Isp

def SpecificImpulse (c,Gravity):
    Isp=c/Gravity
    return Isp

def ExhaustVmass(TotalThrust , PThrust , mdot):
    v=(TotalThrust-PThrust)/mdot
    return v

def CfIdeal(k,PR):
    Cf=(((2*k**2)/(k-1))*(2/(k+1))**((k+1)/(k-1))*(1-(PR)**((k-1)/k)))**0.5
    return Cf

def IdealExitVelocity (k,R,T1,PR):
    v2=(((2*k)/(k-1))*R*T1*(1-((PR)**-1)**((k-1)/k)))**(1/2)
    return v2

def VelocityThroat(k,R,Tth):
    vt=np.sqrt(k*R*Tth)
    return vt

def FindEpsln(numbers,eps):   
     numbers = np.asarray(numbers) 
     i=(np.abs(numbers - eps)).argmin() 
     return numbers[i]  

def AreaThroat (mdot,p1,R,T1,k):
    At=(mdot/p1)*((R*T1)/(k*(2/(k+1))**((k+1)/(k-1))))**(1/2)
    return At

def AreaThroatThrust (F,CF,P1):
    At=F/(CF*P1)
    return At

def FuelConsumptionRatio (Thrust,WFR):
    FCR=WFR/Thrust
    return FCR



In [3]:
#Conversion Factors
#Taken from Appendix 1 "Conversion Factors" pg.743
#designated: cf|code|number
#example: cfacc1

# Acceleration | Lt^2 | acc
def acc():
    m_sec2=3.2808*ft_sec2
    m_sec2=39.3701*in_sec2
    go=9.80665*m_sec2
    ft_sec2=0.3048*m_sec2
    ft_sec2=12*in_sec2
    go=32.174*ft_sec2

# Area | L^2| are
def are():
    m2=1550*in2
    m2=10.7639*ft2
    ft2=144*in2
    ft2=0.092903*m2
    in2=6.4516*10**-4*m2

# Density | ML^3 | den
def den():
    kg_m3=6.24279*10**-2*lbm_ft3
    kg_m3=3.61273*10**-5*lbm_in3
    lbm_ft3=16.0184*kg_m3
    lbm_in3=2.76799*10**4*kg_m3
    #Specific Gravity is dimensionless, same value as density

# Energy/Work/Heat | ML^2t^-2 | ene

# Force | MLt^-2 | frc
def frc():
    kg=9.80665*N #Force in Europe
    ton=kg*1000 #Force in Europe
    N=0.2248089*lbf
    lbf=4.448221*N

# Length/Distance | L | dis
def dis():
    m=3.2808*ft
    ft=0.3048*m
    ft=12*inch
    inch=2.54*cm
    inch=0.0254*m
    mile=1.609344*km
    mile=1609.344*m
    mile=5280*ft
    ntmile=1852*m

# Mass | M | mas
def mas():
    kg=2.205*lbm
    slug=32.174*lbm
    lbm=0.4536*kg

# Power | ML^2t^-3 | pow

# Pressure | ML^-1t^-2 | pre
def pre():
    bar=Pa*10**5
    atm=0.101325*MPa
    atm=14.696*psia
    MPa=Pa*10**6
    psi=6894.757*Pa
    #psi=lbf/in**2
    #bar=0.10*MPa
    #Pa=N/m**2

# Velocity/Speed | Lt^-1 | vel
def vel():
    m_sec=3.2808*ft_sec
    ft_sec=0.3048*m_sec
    knot=0.5144*m/sec
    mile_hr=0.477*m_sec

# Specific Heat | L^2t^-2T^-1 | sph

# Temperature | T | tem
def tem():
    K=(9/5)*R
    R=(5/9)*K
    C=(5/9)*(F-32)
    F=(9/5)*C+32
    #0C=273.15K
    #0F=459.67R

# Time | t | tim

# Viscosity | ML^-1t^-1| vis


# A)
## Redo Example 6.1 using this program and calculate the following design parameters:
- V2
- Ath
- A2
- M2
- Total Weight Flowrate, Wdot
- Oxidizer weight flowrate, Wdoto
- Fuel weight flowrate, Wdotf
- Total propellent weight required, Wp
- Total oxidizer weight required, Wo
- Total fuel weight required, Wf
- Oxidizer "volume flowrate" required, Vdoto
- Fuel "volume flowrate" required, Vdotf
- Total Oxidizer volume required, Vo
- Total Fuel volume required, Vf


In [33]:
#Definitions

def IdealExitVelocity (go,k,R,T1,P2,P1,MW):
    V2=np.sqrt((((2*go*k)/(k-1))*((R*T1)/MW))*(1-((P2/P1)**((k-1)/k))))
    return V2
               
def SpecificImpulse (V2,go):
    Isp=V2/go
    return Isp
               
def CFIdeal(k,P1,P2):
    CF=np.sqrt(((2*k**2)/(k-1))*(2/(k+1))**((k+1)/(k-1))*(1-(P1/P2)**((1-k)/k)))
    return CF

def AreaThroatThrustActual(F,CF_A,P1):
    At=F/(CF_A*P1)
    return At
      
def AreaRatio(k,P1,P2):
    ε=((((k+1)/2)**(1/(k-1)))*((P2/P1)**(1/k))*np.sqrt(((k+1)/(k-1))*(1-(P2/P1)**((k-1)/k))))**(-1)
    return ε

def AreaRatioMach(M2,k):
    εt=(1/M2)*((1+((k-1)/2)*M2**2)/((k+1)/2))**((k+1)/(2*(k-1)))
    return εt

def AreaExitIdeal(At,ε):
    A2=At*ε
    return A2

def WeightFlowrate(F,Isp_A):
    Wdot=F/Isp_A
    return Wdot

def WeightRequiredPropellent(Wdot,OT):
    WRP=Wdot*OT
    return WRP

def WeightFlowrateOx(Wdot,r):
    WdotOx=(Wdot*r)/(r+1)
    return WdotOx

def WeightFlowrateFuel(Wdot,r):
    WdotF=Wdot/(r+1)
    return WdotF

def WeightRequiredOx(WdotOx,OT):
    WROx=WdotOx*OT
    return WROx

def WeightRequiredFuel(WdotF,OT):
    WRF=WdotF*OT
    return WRF

def VolumeFlowrateOx(WdotOX,ρo):
    VdotOx=WdotOx/ρo
    return VdotOx

def VolumeFlowrateFuel(WdotF,ρf):
    VdotF=WdotF/ρf
    return VdotF

def VolumeRequiredOx(VdotOx,OT):
    VROx=VdotOx*OT
    return VROx

def VolumeRequiredFuel(VdotF,OT):
    VRF=VdotF*OT
    return VRF
               
#Given
F=10000     #Thrust | lbf
P1=1000     #Chamber Pressure | psia
r=3.4       #Mixture Ratio | unitless
MW=8.9      #Molecular Weight | lbm/lb-mol
T1=4840     #Chamber Temperature | ⁰R
k=1.26      #Specific Heat Ratio | unitless
P2=1.58     #Exit Pressure | psia
P3=P2       #Optimum Conditions
IspCF=.97   #Specific Impulse Correction Factor
CfCF=.98    #Coefficient of Thrust Correction Factor
R=1544      #Universal Gas Constant | ft.Lbf/Slug*°R
go=32.2     #Gravity | ft/s^2
P1_P2=P1/P2 #Pressure Ratio | Unitless
ρo=71.1     #Weight Density of Oxygen (Oxidizer) | lbf/ft^3
ρf=4.4      #Weight Density of Hydrogen (Fuel) | lbf/ft^3
OT=152      #Operation Time | seconds

             
    
#Exit Velocity
V2=IdealExitVelocity (go,k,R,T1,P2,P1,MW)

##Solver
##Calculating the Optimum Mach Number at Nozzle Exit | Unitless
M2=1      #Setting a test number to 1
res=0.001 #Setting the resolution of the calcualtion increase | Unitless
εt=ε-1    #Initial Value of Area Ratio Modified | Unitless
##Looping the Area Ratio Equation to solve for Mach at Exit
while (εt < ε):
    εt=AreaRatioMach(M2,k) #Equation 3-14 | Ideal Nozzle Area Ratio | Unitless
    M2=M2+res
    


#Ideal Specific Impulse
Isp=SpecificImpulse (V2,go)
#print("Ideal Specific Impulse:\n\t",int(Isp),"seconds \n")
    
#Actual Specific Impulse
Isp_A=Isp*IspCF
#print("Actual Specific Impulse:\n\t",int(Isp_A),"seconds \n")

#Ideal Coefficient of Thrust
CF=CFIdeal(k,P1,P2)
#print("Ideal Coefficient of Thrust:\n\t",round(CF,2))
               
#Actual Coefficient of Thrust
CF_A=CF*CfCF
#print("Actual Coefficient of Thrust:\n\t",round(CF_A,2))

#Throat Area
At=AreaThroatThrustActual(F,CF_A,P1)
              
#Area Ratio
ε=AreaRatio(k,P1,P2)
#print("Area Ratio:\n\t",round(ε))

#Exit Area
A2=AreaExitIdeal(At,ε)
               
#Total Weight Flowrate
Wdot=WeightFlowrate(F,Isp_A)

#Total Propellent Weight
WRP=WeightRequiredPropellent(Wdot,OT)

#Oxidizer Weight Flowrate
WdotOx=WeightFlowrateOx(Wdot,r)

#Oxidizer Weight Required
WROx=WeightRequiredOx(WdotOx,OT)

#Fuel Weight Flowrate
WdotF=WeightFlowrateFuel(Wdot,r)

#Fuel Weight Required
WRF=WeightRequiredFuel(WdotF,OT)

#Oxidizer Volume Flowrate
VdotOx=VolumeFlowrateOx(WdotOx,ρo)

#Oxidizer Volume Required
VROx=VolumeRequiredOx(VdotOx,OT)

#Fuel Volume Flowrate
VdotF=VolumeFlowrateFuel(WdotF,ρf)

#Fuel Volume Required
VRF=VolumeRequiredFuel(VdotF,OT)

#Deliverables
print("Exit Velocity:\n\t",int(V2),"ft/s or",int(V2/3.2808),"m/s \n")
print("Throat Area:\n\t",round(At,1),"in^2")
print("Ideal Nozzle Exit Area:\n\t",round(A2,1),"in^2 or",round(A2/3.2808,7),"m^2 \n")
print("Ideal Exit Mach Number:\n\t",round(M2,4))
print("Total Weight Flowrate:\n\t",round(Wdot,1),"Lbf/sec")
print("Oxidizer Weight Flowrate:\n\t",round(WdotOx,1),"Lbf/sec")
print("Fuel Weight Flowrate:\n\t",round(WdotF,2),"Lbf/sec")
print("Total Propellent Weight Required:\n\t",round(WRP,2),"Lbf")
print("Oxidizer Weight Required for 152 seconds of Operation:\n\t",round(WROx,3),"Lbf")
print("Fuel Weight Required for 152 seconds of Operation:\n\t",round(WRF,3),"Lbf")
print("Oxidizer Volume Flowrate:\n\t",round(VdotOx,3),"Ft^3/sec")
print("Fuel Volume Flowrate:\n\t",round(VdotF,3),"Ft^3/sec")
print("Oxidizer Volume Required for 152 seconds of Operation:\n\t",round(VROx,3),"Ft^3")
print("Fuel Volume Required for 152 seconds of Operation:\n\t",round(VRF,3),"Ft^3")


Exit Velocity:
	 13885 ft/s or 4232 m/s 

Throat Area:
	 5.8 in^2
Ideal Nozzle Exit Area:
	 239.3 in^2 or 72.9288952 m^2 

Ideal Exit Mach Number:
	 4.63
Total Weight Flowrate:
	 23.9 Lbf/sec
Oxidizer Weight Flowrate:
	 18.5 Lbf/sec
Fuel Weight Flowrate:
	 5.43 Lbf/sec
Total Propellent Weight Required:
	 3633.77 Lbf
Oxidizer Weight Required for 152 seconds of Operation:
	 2807.912 Lbf
Fuel Weight Required for 152 seconds of Operation:
	 825.856 Lbf
Oxidizer Volume Flowrate:
	 0.26 Ft^3/sec
Fuel Volume Flowrate:
	 1.235 Ft^3/sec
Oxidizer Volume Required for 152 seconds of Operation:
	 39.492 Ft^3
Fuel Volume Required for 152 seconds of Operation:
	 187.695 Ft^3


In [ ]:
#Definitions

def IdealExitVelocity (k,R,T1,P2,P1,MW):
    V2=np.sqrt(((2*k*R*T1)/((k-1)*MW))*(1-((P2/P1)**((k-1)/k))))
    return V2

def VelocityThroat(k,R,T1,MW):
    Vt=np.sqrt(2*k*R*T1/(MW*(k+1)))
    return Vt

def PressureRatio(P1,P2):
    PR=P2/P1
    return PR

def AreaRatio(k,P1_P2):
    ε=((((k+1)/2)**(1/(k-1)))*((1/P1_P2)**(1/k))*np.sqrt(((k+1)/(k-1))*(1-(1/P1_P2)**((k-1)/k))))**(-1)
    return ε

def AreaRatioMach(M2,k):
    εt=(1/M2)*((1+((k-1)/2)*M2**2)/((k+1)/2))**((k+1)/(2*(k-1)))
    return εt

def AreaThroatIdeal (F,CF,P1):
    At=F/(CF*P1)
    return At

def AreaThroatActual(F,CF_A,P1):
    At_A=F/(CF_A*P1)
    return At_A

def AreaExitIdeal(At,ε):
    A2=At*ε
    return A2

def AreaExitActual(At_A,ε):
    A2_A=At_A*ε
    return A2_A

def CFIdeal(k,P1,P2):
    CF=np.sqrt(((2*k**2)/(k-1))*(2/(k+1))**((k+1)/(k-1))*(1-(P1/P2)**((1-k)/k)))
    return CF

def SpecificImpulse(V2,go):    
    Isp=V2/go    
    return Isp

def TemperatureExit(T1,P1,P2,k):
    T2=T1*(P1/P2)**((1-k)/k)
    return T2

#Given
FI=4.448221*25000  #Thrust| lbf
F=25000           #Thrust | N
P1I=1000           #Chamber Pressure | psia
P1=P1I*6894.76    #Chamber Pressure |Pa
VCF= 0.97          #Velocity Conversion Factor
TCF=0.98           #Temperature Conversion Factor
RI=1544            #Universal Gas Constant | ft.Lbf/Slug*°R
R=8314.3          #Universal Gas Constant | J/kg-mol-K
P2I=14.7           #Sea-Level Pressure | psia
P2=101325         #Sea-Level Pressure | Pa
goI=32.2           #Gravity | ft/sec^2
go=9.8            #Gravity | m/sec^2
P1_P2=P1/P2

#Table 5.5 for Oxy-Meth Mix 
MR=3.20     #Mass Ratio | Unitless
SG=0.81     #Specific Gravity | Unitless
T1=3526     #Chamber Temperature | Kelvin
Cstar=1835  #Chamber Velocity | m/sec
MW=20.3     #Molecular Weight | kg/mol
Isp=296     #Isp Frozen | seconds
k=1.20      #Ratio of Specifc Heats | Unitless


#Calculations

#Exit Velocity
V2=IdealExitVelocity (k,R,T1,P2,P1,MW)
print("Exit Velocity:\n\t",round(V2,2),"m/s or",round(V2*3.2808,2),"ft/s \n")

#Velocity at Throat
Vt=VelocityThroat(k,R,T1,MW)
print("Velocity at Throat:\n\t",round(Vt,2),"m/s or",round(Vt*3.2808,2),"ft/s \n")

#Ideal Nozzle Exit Area
ε=AreaRatio(k,P1_P2)
CF=CFIdeal(k,P1,P2)
At=AreaThroatIdeal(F,CF,P1)
A2=At*ε
print("Ideal Nozzle Exit Area:\n\t",round(A2,7),"m^2 or",round(A2/3.2808,7),"ft^2 \n")

#Ideal Specific Impulse 
Isp=SpecificImpulse(V2,go)
print("Ideal Specific Impulse:\n\t",round(Isp,2),"seconds \n")

#Actual Specific Impulse
Isp_A=Isp*VCF
print("Actual Specific Impulse:\n\t",round(Isp_A,2),"seconds \n")

#Ideal Coefficient of Thrust
CF=CFIdeal(k,P1,P2)
print("Ideal Coefficient of Thrust:\n\t",round(CF,4))

#Actual Coefficient of Thrust
CF_A=CF*TCF
print("Actual Coefficient of Thrust:\n\t",round(CF_A,4))

#Actual Nozzle Throat Area
At_A=AreaThroatActual(F,CF_A,P1)
print("Actual Nozzle Throat Area:\n\t",round(At_A,4),"m^2 or",round(At_A/3.2808,4),"ft^2 \n")

#Nozzle Area Ratio
ε=AreaRatio(k,P1_P2)
print("Nozzle Area Ratio:\n\t",round(ε,4))

#Actual Nozzle Exit Area
A2_A=AreaExitActual(At_A,ε)
print("Actual Nozzle Exit Area:\n\t",round(A2_A,4),"m^2 or",round(A2_A/3.2808,4),"ft^2 \n")

#Ideal Exit Mach Number

##Solver
##Calculating the Optimum Mach Number at Nozzle Exit | Unitless
M2=1      #Setting a test number to 1
res=0.001 #Setting the resolution of the calcualtion increase | Unitless
εt=ε-1    #Initial Value of Area Ratio Modified | Unitless
##Looping the Area Ratio Equation to solve for Mach at Exit
while (εt < ε):
    εt=AreaRatioMach(M2,k) #Equation 3-14 | Ideal Nozzle Area Ratio | Unitless
    M2=M2+res
    
print("Ideal Exit Mach Number:\n\t",round(M2,4))

#Ideal Exit Temperature
T2=TemperatureExit(T1,P1,P2,k)
print("Ideal Exit Temperature:\n\t",round(T2,2),"°K or",round(T2*(9/5),2),"°R\n")




In [ ]:
#Defininitions

def AreaRatio(k,P1_P2):
    ε=((((k+1)/2)**(1/(k-1)))*((1/P1_P2)**(1/k))*np.sqrt(((k+1)/(k-1))*(1-(1/P1_P2)**((k-1)/k))))**(-1)
    return ε

def AreaRatioMach(M2,k):
    εt=(1/M2)*((1+((k-1)/2)*M2**2)/((k+1)/2))**((k+1)/(2*(k-1)))
    return εt

def TempRatio(P1_P3,k):
    TR=P1_P3**((k-1)/k)
    return TR



#Given Values
k=1.23
P1_P3=200   #Pressure Ratio | Unitless
PR=P1_P3 #Ideal Operation Conditions

#Solver
##Calculating the Area Ratio | Unitless
ε=AreaRatio(k,PR)
print("The Nozzle Area Ratio is:\n \t", round(ε,3),"\n")

##Calculating the Optimum Mach Number at Nozzle Exit | Unitless
M2=1      #Setting a test number to 1
res=0.001 #Setting the resolution of the calcualtion increase | Unitless
εt=ε-1    #Initial Value of Area Ratio Modified | Unitless

##Looping the Area Ratio Equation to solve for Mach at Exit
while (εt < ε):
    εt=AreaRatioMach(M2,k) #Equation 3-14 | Ideal Nozzle Area Ratio | Unitless
    M2=M2+res
print("The Optimum Mach Number at Nozzle Exit is:\n\t", round(M2,3),"\n")

#Calculating the Ideal Temperature Ratio between chamber and atmo. | Unitless
TR=TempRatio(P1_P3,k)
print("The Ideal Temperature Ratio (T1/T3) is:\n\t", round(TR,3))

In [ ]:
def AreaRatioMach(M2,k):
    εt=(1/M2)*((1+((k-1)/2)*M2**2)/((k+1)/2))**((k+1)/(2*(k-1)))
    return εt

def PressureExit(P1,k,M2):
    P2=P1*(1+0.5*(k-1)*M2**2)**(-k/(k-1))
    return P2

def TempThroat(T1,k):
    Tt=2*T1/(k+1)
    return Tt

def TempExit(T1,P1,P2,k):
    T2=T1*(P1/P2)**((1-k)/k)
    return T2

def VelocityThroat (k,R,Tt):
    Vt=np.sqrt(k*R*Tt)
    return Vt

def VelocityExit(M2,k,R,T2):
    V2=M2*np.sqrt(k*R*T2)
    return V2

def VelocityExitMax(R,k,T1):
    V2MAX=np.sqrt((2*R*k*T1)/(k-1))
    return V2MAX



#Given
ε=45    #Area Ratio
P1=1300 #Chamber Pressure | psia
T1=2100 #Chamber Temperature | °R 
k=1.27  #Specific Heat Ratio | Unitless
R=1716  #Universal Gas Constant | ft.Lbf/Slug*°R

#Solver
#Calculating the Optimum Mach Number at Nozzle Exit | Unitless
M2=1      #Setting a test number to 1
res=0.001 #Setting the resolution of the calcualtion increase | Unitless
εt=ε-1    #Initial Value of Area Ratio Modified | Unitless

#Looping the Area Ratio Equation to solve for Mach at Exit
while (εt < ε):
    εt=AreaRatioMach(M2,k) #Equation 3-14 | Ideal Nozzle Area Ratio | Unitless
    M2=M2+res
print("Optimum Mach Number at Nozzle Exit is:\n\t", round(M2,3),"\n")

P2=PressureExit(P1,k,M2)

T2=TempExit(T1,P1,P2,k)
print("Static Temperature at the Nozzle Exit:\n\t", round(T2,3),"°R or", round(T2*5/9,2), "°K\n")

print("Static Pressure at the Nozzle Exit:\n\t", round(P2,3),"psi or", round(P2*6894.757,2), "Pa\n")


Tt=TempThroat(T1,k)
Vt=VelocityThroat(k,R,Tt)
print("Velocity at the Throat:\n\t", round(Vt,3), "ft/s or", round(Vt/3.2808,1), "m/s\n")

V2=VelocityExit(M2,k,R,T2)
print("Velocity at the Nozzle Exit Plane:\n\t", round(V2,3), "ft/s or", round(V2/3.2808,1), "m/s\n")

V2MAX=VelocityExitMax(R,k,T1)
print("Theoretical Maximum Velocity at the Nozzle Exit Plane:\n\t", round(V2MAX,1),"ft/s or", round(V2MAX/3.2808,1), "m/s")